In [1]:
# Generar representaciones tradicionales:
# - TF-IDF de palabras.
# - TF-IDF de n-gramas de caracteres.
# Usaremos solo TRAIN para ajustar y transformaremos TRAIN y VALIDATION.
# Guardamos matrices .npz, vectorizers .pkl y un índice con doc_id/sent_id.

#### ***Imports y config***

In [2]:
from pathlib import Path
import pandas as pd
import numpy as np
import pickle, json
from scipy import sparse
from sklearn.feature_extraction.text import TfidfVectorizer

pd.set_option("display.max_colwidth", 120)
SEED = 42

#### ***Rutas***

In [3]:
def find_root():
    p = Path.cwd()
    for cand in [p, *p.parents]:
        if (cand / "data" / "processed").exists():
            return cand
    raise FileNotFoundError("No encuentro data/processed.")

ROOT = find_root()
PROC = ROOT / "data" / "processed"
FEAT = ROOT / "features" / "tfidf"
FEAT.mkdir(parents=True, exist_ok=True)

NIVELES = ["easy","medium","hard"]
SPLITS = ["train","validation"]

#### ***Carga de processed → DataFrame***

In [ ]:
def cargar_processed():
    filas = []
    for level in NIVELES:
        for split in SPLITS:
            p = PROC / level / split / "sentences.jsonl"
            if not p.exists():
                continue
            df = pd.read_json(p, lines=True)
            df["level"] = level
            df["split"] = split
            filas.append(df[["doc_id","sent_id","level","split","text_norm"]])
    return pd.concat(filas, ignore_index=True)

df = cargar_processed().sort_values(["level","split","doc_id","sent_id"]).reset_index(drop=True)
print(df.shape, "filas")
df.head(3)

(208160, 5) filas


,doc_id,sent_id,level,split,text_norm
0,problem-1,0,easy,train,there s also incidents of testosterone insensitive males that have either ambiguous genitals or are phenotypically f...
1,problem-1,1,easy,train,female is the human default body plan so a number of conditions exist that cause female looking males
2,problem-1,2,easy,train,what about people born appearing num female complete with the bits but are genetically males


#### ***TF-IDF de palabras***

In [5]:
# Ajuste en TRAIN
df_train = df[df["split"] == "train"]
corpus_train = df_train["text_norm"].astype(str).tolist()

vec_word = TfidfVectorizer(
    lowercase=False,          # ya normalizado
    analyzer="word",
    ngram_range=(1,2),        # unigrams + bigrams
    min_df=5,                 # filtra ruido
    max_features=100_000
)
Xtr_w = vec_word.fit_transform(corpus_train)

# Transform VALIDATION
df_val = df[df["split"] == "validation"]
Xva_w = vec_word.transform(df_val["text_norm"].astype(str).tolist())

print("WORD TF-IDF")
print("train:", Xtr_w.shape, "nnz:", Xtr_w.nnz)
print("val  :", Xva_w.shape, "nnz:", Xva_w.nnz)

WORD TF-IDF
train: (171602, 85932) nnz: 4449125
val  : (36558, 85932) nnz: 937263


#### ***TF-IDF de n-gramas de caracteres***

In [6]:
vec_char = TfidfVectorizer(
    analyzer="char",
    ngram_range=(3,5),
    min_df=5,
    max_features=200_000
)
Xtr_c = vec_char.fit_transform(corpus_train)
Xva_c = vec_char.transform(df_val["text_norm"].astype(str).tolist())

print("CHAR TF-IDF")
print("train:", Xtr_c.shape, "nnz:", Xtr_c.nnz)
print("val  :", Xva_c.shape, "nnz:", Xva_c.nnz)

CHAR TF-IDF
train: (171602, 134419) nnz: 45449143
val  : (36558, 134419) nnz: 9754529


#### ***Guardado de matrices y metadatos***

In [7]:
# Índices para mapear filas → (level, split, doc_id, sent_id)
idx_train = df_train[["level","split","doc_id","sent_id"]].reset_index(drop=True)
idx_val   = df_val[["level","split","doc_id","sent_id"]].reset_index(drop=True)

# Carpetas
( FEAT / "word" ).mkdir(parents=True, exist_ok=True)
( FEAT / "char" ).mkdir(parents=True, exist_ok=True)

# WORD
sparse.save_npz(FEAT / "word" / "X_train_word.npz", Xtr_w)
sparse.save_npz(FEAT / "word" / "X_val_word.npz",   Xva_w)
pickle.dump(vec_word, open(FEAT / "word" / "vectorizer_word.pkl","wb"))
idx_train.to_csv(FEAT / "word" / "index_train.csv", index=False)
idx_val.to_csv(  FEAT / "word" / "index_val.csv",   index=False)

# CHAR
sparse.save_npz(FEAT / "char" / "X_train_char.npz", Xtr_c)
sparse.save_npz(FEAT / "char" / "X_val_char.npz",   Xva_c)
pickle.dump(vec_char, open(FEAT / "char" / "vectorizer_char.pkl","wb"))
idx_train.to_csv(FEAT / "char" / "index_train.csv", index=False)
idx_val.to_csv(  FEAT / "char" / "index_val.csv",   index=False)

# Informe JSON mínimo
reporte = {
    "word": {
        "shape_train": Xtr_w.shape, "nnz_train": int(Xtr_w.nnz),
        "shape_val":   Xva_w.shape, "nnz_val":   int(Xva_w.nnz),
        "vocab_size":  len(vec_word.vocabulary_)
    },
    "char": {
        "shape_train": Xtr_c.shape, "nnz_train": int(Xtr_c.nnz),
        "shape_val":   Xva_c.shape, "nnz_val":   int(Xva_c.nnz),
        "vocab_size":  len(vec_char.vocabulary_)
    }
}
(Path(FEAT) / "tfidf_resumen.json").write_text(json.dumps(reporte, indent=2), encoding="utf-8")
print("Guardado en features/tfidf/")

Guardado en features/tfidf/


#### ***Tests FInales***

In [8]:
# Terminos más pesados por TF-IDF medio en TRAIN (palabras)
col_means = np.asarray(Xtr_w.mean(axis=0)).ravel()
top_idx = col_means.argsort()[-10:][::-1]
inv_vocab = {j:i for i,j in vec_word.vocabulary_.items()}
top_terms = [inv_vocab[i] for i in top_idx]
pd.DataFrame({"term": top_terms, "tfidf_mean": col_means[top_idx].round(6)})

,term,tfidf_mean
0,the,0.034300
1,to,0.026840
2,and,0.022177
3,of,0.020698
4,in,0.018026
5,it,0.017368
6,is,0.017342
7,that,0.016891
8,they,0.013748
9,you,0.013222


Va bien. Resultados correctos y suficientes para E2. Char-ngrams densos pero ok; word TF-IDF dominado por funcionales, coherente con señal de estilo. 

# Informe breve — `02_representaciones_tradicionales.ipynb`

## Objetivo

Crear TF-IDF de palabras y de n-gramas de caracteres desde `processed` para baselines y modelos. 

## Resultados

**WORD TF-IDF**

* Train: `(171602, 85932)`, nnz `4,449,125`
* Val: `(36558, 85932)`, nnz `937,263`
* Top términos por TF-IDF medio: function words en inglés (`the, to, and…`). 

**CHAR TF-IDF (3–5)**

* Train: `(171602, 134419)`, nnz `45,449,143`
* Val: `(36558, 134419)`, nnz `9,754,529` 

## Validaciones

* Ajuste solo en **train** y transformación en **train/validation**.
* Índices guardados para mapear filas ↔ `(level, split, doc_id, sent_id)`.
* Vectorizadores y matrices persistidos en `features/tfidf/{word,char}/`. 

## Recomendaciones opcionales

* Reducir memoria en CHAR: `dtype=np.float32` y/o `min_df=10`.
* Probar `sublinear_tf=True` para atenuar ráfagas en n-gramas.
* Mantener **sin** stopwords en WORD para preservar señal estilística.
* Si falta RAM, bajar `max_features` a 60–80k (word) y 100–120k (char). 

## Salidas guardadas

* `features/tfidf/word/`: `X_train_word.npz`, `X_val_word.npz`, `vectorizer_word.pkl`, `index_{train,val}.csv`
* `features/tfidf/char/`: `X_train_char.npz`, `X_val_char.npz`, `vectorizer_char.pkl`, `index_{train,val}.csv`
* `features/tfidf/tfidf_resumen.json` (shapes, nnz, vocab). 